In [1]:


'''现在的问题是要将这些步骤连接起来:
    1. 打开某一天的第一版页面，并爬取当天所有版面的链接，保存在list中
    2. 写loop，爬取每个版面的文章列表链接，保存在list中
    3. 将列表链接和网站general链接组合起来，保存在list中
    4. 写loop，爬取当天报纸每一天的文章
    
    '''
'''第一步：爬取当天所有版面的链接，保存在list中'''
#准备好软件包
from bs4 import BeautifulSoup
import urllib.request
import re
import time
import os
import http
from random import uniform

#提取当天第一版的 URL 的HTML内容


a='12'#
bb=[]
for b0 in range(1,32):#only 30 days in April; print bb to check if it is correct 
    if b0<10:
        b0='0'+str(b0)
        bb.append(b0)
    else:
        b0=str(b0)
        bb.append(b0)
    
    
    
for b in bb [:32]: 
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
    general=r'http://paper.people.com.cn/rmrb/html/2024-'+a+'/'+b+'/'  #+a Apil +b month and days
    add_g='nbs.D110000renmrb_01.htm'
    
    newpath = r'C:\Users\Yan\Desktop\EPU\RMB\2024\2024'+a+b
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    os.chdir(newpath)
    
    
    
    try:
        f=urllib.request.urlopen(general+add_g)
        response=f.read()
    #用BeautifulSoup解析数据，'html.parser'在python3中是必须
        html=BeautifulSoup(response,'html.parser')
    except urllib.error.HTTPError:
        print('urllib.error.HTTPError')  
        #出现错误，停几秒先    
    except http.client.IncompleteRead:
        print('IncompleteRead')
        time.sleep(2)
        f=urllib.request.urlopen(general+add_g)
        response=f.read()
    
    #提取版面链接
    pagelink=html.find_all(id='pageLink')
    res_p=r"(?<=href=\").+?(?=\")|(?<=href=\').+?(?=\')"
    res_pp=re.compile(res_p) #the regular expression to find the URL
    links=[]
    for i in pagelink:
        page=res_pp.findall(str(i))
        links.append(page)
    
    links=[item for sublist in links for item in sublist]
    len(links)
    
    '''2. 写loop，爬取每个版面的文章列表链接，保存在list中'''
    
    #提取该版面上每篇文章的链接
    page_links=[]
    for i in links:
        page_link=general+i
        page_links.append(page_link)
        
    len(page_links)
    
    
    
    #用while，try和exception自动处理错误
    title_lists=[]
    m=0
    while m<len(page_links):
        try:
            f=urllib.request.urlopen(page_links[m])
            response=f.read()
            html=BeautifulSoup(response,'html.parser')
            title_list= html.find_all('div',attrs={'class':'news'})
            titles=res_pp.findall(str(title_list))
            title_lists.append(titles)
            m=m+1
        except http.client.IncompleteRead:
            print('IncompleteRead')
            time.sleep(2)
            m=m
        except urllib.error.HTTPError:
            print('urllib.error.HTTPError')  
            time.sleep(2)#出现错误，停几秒先
            m=m+1
        except http.client.RemoteDisconnected:
            print('RemoteDisconnected')
            time.sleep(2)
            m=m
            
    len(title_lists)
    
    if len(page_links)!=len(title_lists):
        print('Alert')
    else:
        print('Congrats,pass!')
        
    '''3. 将列表链接和网站general链接组合起来，保存在list中'''
    title_links=[]
    for i in title_lists:
        for j in i:
            title_link=general+j
            title_links.append(title_link)
            
    len(title_links)
    
    '''4. 写loop，爬取当天报纸每一天的文章'''
    
    
    
    ##设置休息时间，以免把server搞摊了
    #del title_links[3]
    
    m=0
    while m<len(title_links):
        try:
            gmrb_request=urllib.request.Request(url=title_links[m], headers=headers)
            gmrb_response=urllib.request.urlopen(gmrb_request)
            response=gmrb_response.read()
            html=BeautifulSoup(response,'html.parser')
            title=html.find_all('h1')#找到文章的题目
            date_author=html.find_all('p',attrs={'class':'sec'})#找到文章的作者和时间
            content=html.find_all(id='articleContent')#找到文章的内容
            all_=title+date_author+content
            text_all=''
            for j in all_:
                text=j.get_text()
                text_all=text_all+text
        #print(text_all)
            with open(str(m)+'.txt','w',encoding='utf8') as f:
                f.write(text_all)
            time.sleep(uniform(2,10))
            m=m+1
        except http.client.IncompleteRead:
            print('IncompleteRead')
            time.sleep(uniform(0,2))
            m=m
        except urllib.error.HTTPError:
            print('urllib.error.HTTPError')  
            time.sleep(uniform(1,4))#出现错误，停几秒先
            m=m+10
        except http.client.RemoteDisconnected:
            print('RemoteDisconnected')
            time.sleep(uniform(1,4))
            m=m
    
    
    #import http.client
    #http.client.HTTPConnection._http_vsn = 11
    #http.client.HTTPConnection._http_vsn_str = 'HTTP/1.1'

urllib.error.HTTPError


NameError: name 'html' is not defined